In [78]:
from django.db import connection
from django.contrib.auth.models import User
from myboard import models
from django.shortcuts import render
from django.test import RequestFactory


1. DB browser for slqite3에서 Web>mysite에 있는 db.sqlite3로 데이터베이스 열고 SQL 실행에서 다음을 입력하고 실행시키다.

In [ ]:
CREATE TABLE myboard_image
(
id INTEGER   PRIMARY KEY AUTOINCREMENT,
author_id INTEGER NOT NULL,
filename VARCHAR(50) NOT NULL,
FOREIGN KEY (author_id) REFERENCES auth_user(id)
)

2. mysite>myboard>models.py에 다음의 class 하나를 추가한다.(**불필요**)

In [ ]:
class Image() 어쩌구 저쩌구

3. DB browser for sqlite3에서 `데이터보기`탭의 mybroad_image에서 새레코드를 클릭하고 상단의 값 부분에 author_id는 2, filename은 face01.jpg를 입력했을 때 
    - 하단에 나오는 부분을 복사해서 sql로 삼는다. 
    - 저장하고 마지막에 `변경사항 저정하기`를 누르면 commit까지 완료

In [84]:
username = "home"
sql = f"SELECT id from auth_user where username='{username}'"

cursor = connection.cursor()
cursor.execute(sql)
author_id = cursor.fetchone()[0]
print(author_id)

1


In [85]:
filename = 'face2.png'

sql = f"""
INSERT INTO myboard_image
(author_id, filename)
VALUES ({author_id}, '{filename}');
"""

print(sql)


INSERT INTO myboard_image
(author_id, filename)
VALUES (1, 'face2.png');



In [86]:
v = cursor.execute(sql)

OperationalError: database is locked

In [95]:
def dictfetchall(cursor):
    desc = cursor.description
    return[
        dict(zip([col[0] for col in desc], row )) 
        for row in cursor.fetchall()
    ]

In [96]:
sql = f"""
select filename
from myboard_image
where author_id = (select id from auth_user where username="{username}")
"""

print(sql)


select filename
from myboard_image
where author_id = (select id from auth_user where username="home")



In [97]:
cursor = connection.cursor()
cursor.execute(sql)
data = dictfetchall(cursor)
context = {"data":data, "username":"home"}
print(context)

{'data': [{'filename': 'f1.jpg'}, {'filename': 'f2.jpg'}, {'filename': 'f3.jpg'}, {'filename': 'f4.jpg'}, {'filename': 'f5.jpg'}], 'username': 'home'}


In [98]:
rf = RequestFactory()
url = '/myboard/ajaxget?page=1&category="common"&username="home"'
request = rf.get(url)

response = render(request, "myboard/photolist.html", context)
print(response)
print(response.content.decode())

<HttpResponse status_code=200, "text/html; charset=utf-8">


home <br>

<form action="upload" enctype="multipart/form-data" method="POST">
<input type='hidden' name='csrfmiddlewaretoken' value='Voh9kYje5SYk9XEgXXYQS0fWsnOOLCjb28P5ayTQdFJD5JHm21K8tGnqQeR0QNBp' />
    <input type="file" name="filename"/>
    <input type="submit" value="사진전송">
    <input type='hidden' name="username" value = "home">
</form>



    <img src="/static/face/home/f1.jpg" width="100" 

    <img src="/static/face/home/f2.jpg" width="100" 

    <img src="/static/face/home/f3.jpg" width="100" 

    <img src="/static/face/home/f4.jpg" width="100" 

    <img src="/static/face/home/f5.jpg" width="100" 





In [105]:
print(request.POST["username"])
username = request.POST["username"]

home


In [ ]:
file = request.FILES['file1']
filename = file._name
fp = open(settings.BASE_DIR + "/static/" + filename, "wb")
for chunk in file.chunks() :
    fp.write(chunk)
fp.close()

In [110]:
from django.core.files.uploadedfile import SimpleUploadedFile
f = open("static/people.jpg", "rb")

file_upload = SimpleUploadedFile("people.jpg", f.read(), 
                                content_type="image/jpeg")

data = {"filename": file_upload, "username":"home"}

request = rf.post("/myboard/upload", data=data, format="multipart")

In [111]:
request.POST["username"]

'home'

In [115]:
# 파일 저장
file = request.FILES['filename']
filename = file._name
fp = open(settings.BASE_DIR + "/static/faces/" + username + '/' + filename, "wb")
for chunk in file.chunks() :
    fp.write(chunk)
fp.close()